# Hyperparameter Tuning using HyperDrive

## Import libraries for Azure Machine Learning SDK

In [1]:
import os
import shutil
import numpy as np 
import pandas as pd 
import azureml.core
import pkg_resources
import joblib
import requests
import json

from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

from azureml.core import Environment, Workspace, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

# Check the core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


## Dataset
### Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()

print('Workspace name:\t\t' + ws.name,
      'Resource group:\t\t' + ws.resource_group,
      'Azure region:\t\t'   + ws.location,
      'Subscription id:\t'  + ws.subscription_id, sep='\n')

Workspace name:		quick-starts-ws-237941
Resource group:		aml-quickstarts-237941
Azure region:		southcentralus
Subscription id:	a24a24d5-8d87-4c8a-99b6-91ed2d2df51f


### Create an Azure ML experiment

In [3]:
# Name for experiment
experiment_name = 'hyperdrive-heart-failure-experiment'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

experiment

Name,Workspace,Report Page,Docs Page
hyperdrive-heart-failure-experiment,quick-starts-ws-237941,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute Target
We will need to create a compute target for our AutoML run. We will use ***vm_size = Standard_DS3_v2*** in our provisioning configuration and select ***max_nodes*** to be no greater than 4.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Name for the CPU cluster
amlcompute_cluster_name = "hyperdrive-cpu-compute-cluster"

# Verify that cluster does not exist already
try:
    amlcompute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    amlcompute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', max_nodes=4)
    amlcompute_target = ComputeTarget.create(ws, amlcompute_cluster_name, amlcompute_config)

amlcompute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
compute_targets = ws.compute_targets

for i, key in enumerate(compute_targets):
    print(f"{i+1}. Compute target\n\tname: {compute_targets[key].name}\n\tType: {compute_targets[key].type}")

1. Compute target
	name: notebook237941
	Type: ComputeInstance
2. Compute target
	name: automl-cpu-compute-cluster
	Type: AmlCompute
3. Compute target
	name: hyperdrive-cpu-compute-cluster
	Type: AmlCompute


In [6]:
# For a more detailed view of current AmlCompute status, use get_status().
print(amlcompute_target.get_status().serialize())

{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-06-30T12:58:28.402000+00:00', 'errors': None, 'creationTime': '2023-06-30T12:55:28.015473+00:00', 'modifiedTime': '2023-06-30T12:55:35.073126+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The pipeline we use here consists of a custom logistic regression model of the scikit-learn model stored in the ***train.py*** script and a hyperdrive run that iterates through the model parameters.
- Setting logistic regression parameters:
    - --C - Inverse of regularization strenght
    - --max_iter - Maximum number of iterations convergence

***RandomParameterSampling*** defines random sampling over a hyperparameter search space. This sampling algorithm selects parameter values from a series of discrete values or a distribution over a continuous range. This has an advantage over the GridSearch method, which executes all combinations of parameters and takes a long time to execute.

For the "*Inverse of regularization strength*" parameter, I chose a uniform distribution with min=0.0001 and max=1.0. For the maximum number of iterations convergence, I entered a range of values (5, 25, 50, 100, 200, 500, 1000).

***BanditPolicy*** class Defines a early termination policy based on Slack criteria and a frequency and delay interval for evaluation. This goes a long way towards ensuring that if a model with certain parameters does not work well, it is rejected instead of letting it run longer.

The training takes place on the local computing target. Accuracy was chosen as the primary metric with the goal of maximizing accuracy. To keep the training short, max_total_runs was set to 50 and max_duration_minutes to 20.

***max_concurrent_runs = 4*** : The maximum number of iterations that could be run in parallel. It is recommended to create a dedicated cluster per experiment and adjust the number of max_concurrent_iterations of your experiment to the number of nodes in the cluster. In this way you use all nodes of the cluster at the same time with the desired number of concurrent child runs/iterations. So I set the value to 4.

In [7]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
###early_termination_policy = <your policy here>
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
###param_sampling = <your params here>
param_sampling = RandomParameterSampling(
    {
        "--C": uniform(0.0001, 1.0),
        "--max_iter": choice(5, 25, 50, 100, 200, 500, 1000)
    }
)

script_folder = "./training"

if "training" not in os.listdir():
    os.mkdir(script_folder)
    
shutil.copy('./train.py', script_folder)


'''
#TODO: Create your estimator and hyperdrive config
##estimator = <your estimator here>
estimator = SKLearn(source_directory=script_folder,
                    compute_target=amlcompute_target,
                    entry_script="train.py")

##hyperdrive_run_config = <your config here?
hyperdrive_run_config = HyperDriveConfig(estimator=estimator, 
                                        hyperparameter_sampling=param_sampling, 
                                        policy=early_termination_policy, 
                                        primary_metric_name='Accuracy',
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                        max_total_runs=50,   
                                        max_duration_minutes=20,
                                        max_concurrent_runs=4) 
'''
'''
SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' 
with your own defined environment or the AzureML-Tutorial curated environment.
'''

# Setup environment for the training run
sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=script_folder, 
                      script='train.py', 
                      compute_target=amlcompute_target, 
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(run_config=src, 
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=50,   
                                     max_duration_minutes=20,
                                     max_concurrent_runs=4)  

sklearn_env

{
    "assetId": "azureml://registries/azureml/environments/AzureML-Tutorial/versions/88",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210507.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "AzureML-Tutorial",
    "python": {
        "baseCondaEnvironment

In [8]:
#TODO: Submit your experiment
# Submit the experiment on the configured remote compute cluster
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

In [9]:
# Wait for the cluster to complete, show the output log
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_87e5081a-6e93-4adb-b358-c325923193fc
Web View: https://ml.azure.com/runs/HD_87e5081a-6e93-4adb-b358-c325923193fc?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-237941/workspaces/quick-starts-ws-237941&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-30T14:10:26.788772][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-06-30T14:10:27.2828587Z][SCHEDULER][INFO]Scheduling job, id='HD_87e5081a-6e93-4adb-b358-c325923193fc_0' 
[2023-06-30T14:10:27.4199987Z][SCHEDULER][INFO]Scheduling job, id='HD_87e5081a-6e93-4adb-b358-c325923193fc_1' 
[2023-06-30T14:10:27.5361737Z][SCHEDULER][INFO]Scheduling job, id='HD_87e5081a-6e93-4adb-b358-c325923193fc_2' 
[2023-06-30T14:10:27.6469522Z][SCHEDULER][INFO]Scheduling job, id='HD_87e5081a-6e93-4adb-b358-c325923193fc_3' 
[2023-06-30T14:10:27.592471][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_87e5081a-6e93-4adb-b358-c325923193fc',
 'target': 'hyperdrive-cpu-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-30T14:10:25.887466Z',
 'endTimeUtc': '2023-06-30T14:26:01.85115Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'cbeacc1a-d302-44b7-bf90-3875dea7d3d6',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': 'infinite_space_size',
  'score': '0.9333333333333333',
  'best_child_run_id': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_6',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_87e5081a-6e93-4adb-b358-c325923193fc_6'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

Use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperdrive_run).show()
for children_run in hyperdrive_run.get_children():
    print('-----------------------------------')
    print(children_run)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_87e5081a-6e93-4adb-b358-c325923193fc_49,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_87e5081a-6e93-4adb-b358-c325923193fc_48,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_87e5081a-6e93-4adb-b358-c325923193fc_47,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_87e5081a-6e93-4adb-b358-c325923193fc_45,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_87e5081a-6e93-4adb-b358-c325923193fc_44,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
# evaluate if the the run is indeed complete
assert(hyperdrive_run.get_status() == "Completed")

In [12]:
# Each child in the result has run id, hyperparameters, best primary metric value and status.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

# get the best run and display the properties of the model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print("Best Experiment Run:")
print("-------------------------------------------")
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('Max iterations:',best_run_metrics['Max iterations:'])

best_run

[{'run_id': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_6', 'hyperparameters': '{"--C": 0.031198596838545492, "--max_iter": 500}', 'best_primary_metric': 0.9333333333333333, 'status': 'Completed'}, {'run_id': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_38', 'hyperparameters': '{"--C": 0.24088063216070124, "--max_iter": 500}', 'best_primary_metric': 0.9, 'status': 'Completed'}, {'run_id': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_23', 'hyperparameters': '{"--C": 0.035950992261276085, "--max_iter": 1000}', 'best_primary_metric': 0.9, 'status': 'Completed'}, {'run_id': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_29', 'hyperparameters': '{"--C": 0.8319336708968202, "--max_iter": 100}', 'best_primary_metric': 0.8833333333333333, 'status': 'Completed'}, {'run_id': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_24', 'hyperparameters': '{"--C": 0.9907915478389895, "--max_iter": 200}', 'best_primary_metric': 0.8833333333333333, 'status': 'Completed'}, {'run_id': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_31', 'h

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-heart-failure-experiment,HD_87e5081a-6e93-4adb-b358-c325923193fc_6,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
best_run.get_details()

{'runId': 'HD_87e5081a-6e93-4adb-b358-c325923193fc_6',
 'target': 'hyperdrive-cpu-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-30T14:11:42.688545Z',
 'endTimeUtc': '2023-06-30T14:12:11.047488Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': 'cbeacc1a-d302-44b7-bf90-3875dea7d3d6',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.031198596838545492', '--max_iter', '500'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'hyperdrive-cpu-compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
 

In [14]:
best_run.get_metrics()

{'Regularization Strength:': 0.031198596838545492,
 'Max iterations:': 500,
 'Accuracy': 0.9333333333333333}

In [15]:
for primary_metric in best_run.get_metrics():
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

Regularization Strength: 0.031198596838545492
Max iterations: 500
Accuracy 0.9333333333333333


In [16]:
best_run.get_metrics(name='Accuracy')

{'Accuracy': 0.9333333333333333}

In [17]:
best_run.get_properties()

{'_azureml.ComputeTargetType': 'amlctrain',
 'ContentSnapshotId': 'cbeacc1a-d302-44b7-bf90-3875dea7d3d6',
 'ProcessInfoFile': 'azureml-logs/process_info.json',
 'ProcessStatusFile': 'azureml-logs/process_status.json'}

### Save the best model

In [18]:
best_run.get_file_names()

['logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [19]:
best_run.download_file(name='outputs/model.joblib', output_file_path='./outputs')
best_model = best_run.register_model(model_name='best_run_hyperdrive', model_path='outputs/model.joblib')

In [20]:
from azureml.automl.core.shared import constants

# create inference folder
inference_folder = 'inference'
if inference_folder not in os.listdir():
    os.mkdir(inference_folder)

# Save the best model, scoring script, and conda env files in inference folder
best_run.download_file('outputs/model.joblib', inference_folder + '/best_hyperdrive_model.joblib')

### Save the environment

In [21]:
from azureml.core.environment import Environment

# get the list of environments
Environment.list(workspace=ws).keys()

dict_keys(['AzureML-Triton', 'AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu', 'AzureML-VowpalWabbit-8.8.0', 'AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu', 'AzureML-ACPT-pytorch-1.12-py38-cuda11.6-gpu', 'AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu', 'AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu', 'AzureML-PyTorch-1.3-CPU'])

In [22]:
# save the environment
sklearn_env.save_to_directory('env', overwrite=True)

sklearn_env

{
    "assetId": "azureml://registries/azureml/environments/AzureML-Tutorial/versions/88",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210507.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "AzureML-Tutorial",
    "python": {
        "baseCondaEnvironment

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
# Register the model
from azureml.core.resource_configuration import ResourceConfiguration

model_name = "best_run_hyperdrive"

local_file = inference_folder + '/best_hyperdrive_model.joblib'

run_id = best_run.id
experiment_name = best_run.experiment.name

model = Model.register(workspace = ws,
                       model_name = model_name,                        # Name of the registered model in your workspace.
                       model_path = local_file,                        # Local file to upload and register as a model.
                       model_framework = Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       description = 'Best autoML model to predict motality caused by heart failure.',
                       tags={'area': 'heart-failure', 'type': 'classification'})

print('Model name:', model.name)
print('Model id:', model.id)
print('Model version:', model.version)

Registering model best_run_hyperdrive
Model name: best_run_hyperdrive
Model id: best_run_hyperdrive:4
Model version: 4


In [24]:
# Create a scoring script
%%writefile best_hyperdrive_score.py

import os
import numpy as np
import json
import joblib
from azureml.core.model import Model

def init():
    global model
    try:
        model_path = Model.get_model_path('best_run_hyperdrive')
        model = joblib.load(model_path)
    except Exception as err:
        print("init method error: "+str(err))

def run(data):
    try:
        #data = np.array(json.loads(data))
        data = json.loads(data)
        strn = "extracted json\n"
        data = np.array(data["data"])
        strn += "converted data to np array\n"
        result = model.predict(data)
        strn += "sent the data to the model for prediction\n"
        print(strn)
        return result.tolist()
    except Exception as err:
        return strn+"run method error: "+str(err)

UsageError: Line magic function `%%writefile` not found.


In [ ]:
# create inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

#inference_config = InferenceConfig(entry_script=inference_folder + '/best_hyperdrive_score.py', environment=sklearn_env)
inference_config = InferenceConfig(entry_script='best_hyperdrive_score.py', environment=sklearn_env)

# display the environment file
'''
with open(file_path=inference_folder + 'automl_conda_env.yml', 'r') as file:
    env_file = file.read()
    print(env_file)
'''

In [ ]:
# Model Deployment
from azureml.core.webservice import AciWebservice

# define deployment configuration
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                           memory_gb=1,
                                                           tags={'area': "heart-failure", 'type': "classification"},
                                                           description="Predict heart failure mortality using classification model",
                                                           auth_enabled=True,
                                                           enable_app_insights=True)

# deploy model as webservice using Azure Container Instance(ACI)
aci_service = Model.deploy(workspace = ws, 
                           name = "aci-heart-failure-deploy", 
                           models = [model], 
                           inference_config = inference_config, 
                           deployment_config = aci_deployment_config, 
                           overwrite=True)

aci_service.wait_for_deployment(show_output=True)

In [ ]:
# get the active api endpoint for scoring
print(f"Service State: {aci_service.state}\n")
print(f"Scoring URI:   {aci_service.scoring_uri}\n")
print(f"Swagger URI:   {aci_service.swagger_uri}\n")

### Consuming the model
TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Send a request to the web service
import json
import requests

'''
input_data = json.dumps({
    "data": [
            [75.0, 0.0, 582.0, 0.0, 20.0, 1.0, 265000.0, 1.9, 130.0, 1.0, 0.0, 4.0],
            [80.0, 1.0, 123.0, 0.0, 35.0, 1.0, 388000.0, 9.4, 133.0, 1.0, 1.0, 10.0],
            [62.0, 0.0, 61.0, 1.0, 38.0, 1.0, 155000.0, 1.1, 143.0, 1.0, 1.0, 270.0],
            [50.0, 1.0, 111.0, 0.0, 20.0, 0.0, 210000.0, 1.9, 137.0, 1.0, 0.0, 7.0]
        ]
    })
'''

# 4 sets of data to score, so we get two results back
test_sample = test_df.sample(n=4)
labels = test_sample.pop('DEATH_EVENT')


# Convert to JSON string
input_data = json.dumps({"data": test_sample.to_dict(orient='records')})
with open("input_data.json", 'w') as _f:
    _f.write(input_data)

print(input_data)

response = requests.post(aci_service.scoring_uri, data=input_data, headers={'Content-Type':'application/json'})

In [ ]:
print(f"Predictions from Service: {response.json()}\n")
print(f"Data Labels: {labels.tolist()}")

### Print the logs of the web service and delete the service
TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Print the log of the webservice
print(aci_service.get_logs())

In [ ]:
# Delete the webservice, model, and shut down the compute cluster
#aci_service.delete()
#model.delete()
#amlcompute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

